In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
import os
import re
import numpy as np

%load_ext tensorboard

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Tensorflow/tf-reconigtion/imagenes'
%ls

/content/drive/My Drive/Colab Notebooks/Tensorflow/tf-reconigtion/imagenes
classify_image_graph_def.pb                    logs/              NodeLookup.py
imagenet_2012_challenge_label_map_proto.pbtxt  mittens.JPG        __pycache__/
imagenet_synset_to_human_label_map.txt         mittens_small.jpg


In [ ]:
import NodeLookup as lookup

#Ruta para cargar el modelo
graph_path = os.path.join('classify_image_graph_def.pb')
label_path = os.path.join('imagenet_2012_challenge_label_map_proto.pbtxt')
human_label = os.path.join('imagenet_synset_to_human_label_map.txt')

node_lookup = lookup.NodeLookup(label_lookup_path = label_path, uid_lookup_path = human_label )

#Cargar archivo 
with tf.gfile.FastGFile(graph_path, 'rb') as graphFile:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(graphFile.read())
    _ = tf.import_graph_def(graph_def, name='Imagenet')
    print('Cargando el modelo')

#Decirle a tf que tome la GPU si es necesario
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#Empezar la Session y usar el modelo que se encuentra cargado
with tf.Session(config=config) as session:
    # Saber cual es el modelo
    file_writer = tf.summary.FileWriter('logs', session.graph)
    output_tensor = session.graph.get_tensor_by_name('Imagenet/softmax:0')

    while True:
      image_name = input('Nombre de la imagen')
      if image_name !='q':
        image_data = tf.gfile.FastGFile(image_name, 'rb').read()
        input_tensor_name_data = {
            'Imagenet/DecodeJpeg/contents:0':image_data
        }

        #Matriz de predicciones
        predictions = session.run(output_tensor, input_tensor_name_data)
        print('Multidimensional\n',predictions)
        #Unidimensional
        predictions = np.squeeze(predictions)
        print('Unidimensional\n',predictions)

        #Top 5 predicciones mas altas
        n = 5
        top_predictions = predictions.argsort()[-n:][::-1]
        for prediction in top_predictions:
          percentage = predictions[prediction]
          human_string = node_lookup.id_to_string(prediction)
          #Nombres de clases top 5
          print('{} == {:.2%}'.format(human_string, percentage))

      else:
        break


Cargando el modelo
Nombre de la imagenmittens.JPG
Multidimensional
 [[0.00024689 0.00139867 0.00010926 ... 0.0002469  0.00024689 0.00024689]]
Unidimensional
 [0.00024689 0.00139867 0.00010926 ... 0.0002469  0.00024689 0.00024689]
tabby, tabby cat == 12.61%
tiger cat == 9.85%
Siamese cat, Siamese == 6.28%
Egyptian cat == 6.14%
soccer ball == 5.89%
Nombre de la imagenq
